## Setup


In [ ]:
!pip install transformers
!pip install tokenizers



     |████████████████████████████████| 3.4 MB 3.9 MB/s 
     |████████████████████████████████| 895 kB 36.1 MB/s 
     |████████████████████████████████| 3.3 MB 17.7 MB/s 
     |████████████████████████████████| 596 kB 42.8 MB/s 
     |████████████████████████████████| 61 kB 339 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#import libraries
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig

max_len = 384
configuration = BertConfig()  # default parameters and configuration for BERT

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Set-up BERT tokenizer


In [ ]:
# Save the slow pretrained tokenizer
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
save_path = "bert_base_uncased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer("bert_base_uncased/vocab.txt", lowercase=True)


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Load the data


In [ ]:
train_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json"
train_path = keras.utils.get_file("train.json", train_data_url)
eval_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
eval_path = keras.utils.get_file("eval.json", eval_data_url)


4866048/4854279 [==============================] - 0s 0us/step


## Preprocess the data


In [ ]:

class SquadExample:
    def __init__(self, question, context, start_char_idx, answer_text, all_answers):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.all_answers = all_answers
        self.skip = False

    def preprocess(self):
        context = self.context
        question = self.question
        answer_text = self.answer_text
        start_char_idx = self.start_char_idx

        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        answer = " ".join(str(answer_text).split())

        # Find end character index of answer in context
        end_char_idx = start_char_idx + len(answer)
        if end_char_idx >= len(context):
            self.skip = True
            return

        # Mark the character indexes in context that are in answer
        is_char_in_ans = [0] * len(context)
        for idx in range(start_char_idx, end_char_idx):
            is_char_in_ans[idx] = 1

        # Tokenize context
        tokenized_context = tokenizer.encode(context)

        # Find tokens that were created from answer characters
        ans_token_idx = []
        for idx, (start, end) in enumerate(tokenized_context.offsets):
            if sum(is_char_in_ans[start:end]) > 0:
                ans_token_idx.append(idx)

        if len(ans_token_idx) == 0:
            self.skip = True
            return

        # Find start and end token index for tokens from answer
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        # Tokenize question
        tokenized_question = tokenizer.encode(question)

        # Create inputs
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
        attention_mask = [1] * len(input_ids)

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = max_len - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_context.offsets


with open(train_path) as f:
    raw_train_data = json.load(f)

with open(eval_path) as f:
    raw_eval_data = json.load(f)

#creating exampless set
def create_squad_examples(raw_data):
    squad_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                question = qa["question"]
                answer_text = qa["answers"][0]["text"]
                all_answers = [_["text"] for _ in qa["answers"]]
                start_char_idx = qa["answers"][0]["answer_start"]
                squad_eg = SquadExample(
                    question, context, start_char_idx, answer_text, all_answers
                )
                squad_eg.preprocess()
                squad_examples.append(squad_eg)
    return squad_examples

#creating target
def create_inputs_targets(squad_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x, y

#training data
train_squad_examples = create_squad_examples(raw_train_data)
x_train, y_train = create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")

#testing data
eval_squad_examples = create_squad_examples(raw_eval_data)
x_eval, y_eval = create_inputs_targets(eval_squad_examples)
print(f"{len(eval_squad_examples)} evaluation points created.")


87599 training points created.
10570 evaluation points created.


In [ ]:
x_train[0].shape[0]

86136

Decay LR code

In [ ]:
initial_learning_rate = 5e-5
batchSize = 64
decaySteps=(x_train[0].shape[0])//batchSize
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=decaySteps, decay_rate=0.1, staircase=True)

Create the Question-Answering Model using BERT and Functional API


In [ ]:

def create_model():
    ## BERT encoder
    encoder = TFBertModel.from_pretrained("bert-base-uncased")

    ## QA Model - Stacking 2 NN and a embedding layer to get better result
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding = encoder(
        input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
    )[0]

    start_logits = layers.Dense(1, name="start_logit", use_bias=False)(embedding)
    start_logits = layers.Flatten()(start_logits)

    end_logits = layers.Dense(1, name="end_logit", use_bias=False)(embedding)
    end_logits = layers.Flatten()(end_logits)

    start_probs = layers.Activation(keras.activations.softmax)(start_logits)
    end_probs = layers.Activation(keras.activations.softmax)(end_logits)

    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[start_probs, end_probs],
    )
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = keras.optimizers.Adam(learning_rate=5e-5) #use lr_schedule
    model.compile(optimizer=optimizer, loss=[loss, loss])
    return model



In [ ]:
model = create_model()
model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_3[0][0]',            

In [ ]:
use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()

model.summary()


INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.95.246.50:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.95.246.50:8470


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_3[0][0]',            

## Create evaluation Callback

This callback will compute the exact match score and F-1 score using the validation data
after every epoch.


In [ ]:

f_score_list = []
exact_match_score_list = []

def normalize_text(text):
    text = text.lower()

    # Remove punctuations
    exclude = set(string.punctuation)
    text = "".join(ch for ch in text if ch not in exclude)

    # Remove articles
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    text = re.sub(regex, " ", text)

    # Remove extra white space
    text = " ".join(text.split())
    return text


class FScore(keras.callbacks.Callback):
    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        pred_start, pred_end = self.model.predict(self.x_eval)
        print("start f score")
        f_score = 0.0
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        count = 0.0
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            squad_eg = eval_examples_no_skip[idx]
            prec = 0.0
            rec = 0.0
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else:
                pred_ans = squad_eg.context[pred_char_start:]

            normalized_pred_ans = normalize_text(pred_ans)
            normalized_true_ans_list = [normalize_text(_) for _ in squad_eg.all_answers]
            normalized_pred_ans_tokens = normalized_pred_ans.split()
            count+=1
            temp_f_score = 0
            for normalize_true_ans in normalized_true_ans_list:
              normalize_true_ans_tokens = normalize_true_ans.split()
              if len(normalized_pred_ans_tokens) == 0 or len(normalize_true_ans_tokens) == 0:
                temp_f_score=1
                break
              common_tokens = set(normalized_pred_ans_tokens) & set(normalize_true_ans_tokens)
              # if there are no common tokens then f1 = 0
              if len(common_tokens) == 0:
                continue
              #Precision and Recall calculation
              prec = len(common_tokens) / len(normalized_pred_ans_tokens)
              rec = len(common_tokens) / len(normalize_true_ans_tokens)
              #Picking the max fscore score across the data set
              temp_f_score = max(temp_f_score, 2 * (prec * rec) / (prec + rec))
            f_score+=temp_f_score
        
        #Taking average of max f score over epochs
        f_score = f_score/count
          
        print(f"\nepoch={epoch+1}, f score={f_score:.2f}")
        f_score_list.append(f_score)
        
class ExactMatch(keras.callbacks.Callback):
    """
    Each `SquadExample` object contains the character level offsets for each token
    in its input paragraph. We use them to get back the span of text corresponding
    to the tokens between our predicted start and end tokens.
    All the ground-truth answers are also present in each `SquadExample` object.
    We calculate the percentage of data points where the span of text obtained
    from model predictions matches one of the ground-truth answers.
    """

    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        pred_start, pred_end = self.model.predict(self.x_eval)
        count = 0
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            squad_eg = eval_examples_no_skip[idx]
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else:
                pred_ans = squad_eg.context[pred_char_start:]

            normalized_pred_ans = normalize_text(pred_ans)
            normalized_true_ans = [normalize_text(_) for _ in squad_eg.all_answers]
            #For exact match
            if normalized_pred_ans in normalized_true_ans:
                count += 1
        #Calcualting exact match
        acc = count / len(self.y_eval[0])
        print(f"\nepoch={epoch+1}, exact match score={acc:.2f}")
        exact_match_score_list.append(acc)



## Train and Evaluate


In [ ]:
exact_match_callback = ExactMatch(x_eval, y_eval)
f_score_callback = FScore(x_eval, y_eval)
model.fit(
    x_train,
    y_train,
    epochs=3,  
    verbose=1,
    batch_size=256,
    callbacks=[exact_match_callback, f_score_callback],
)
#save model
model.save('./content/drive/model_bert_squad_256.h5')

Epoch 1/3


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None,) dtype=int64>, <tf.Tensor 'cond/Identity_32:0' shape=(None,) dtype=int64>]


Instructions for updating:
use `experimental_local_results` instead.


Instructions for updating:
use `experimental_local_results` instead.
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None,) dtype=int64>, <tf.Tensor 'cond/Identity_32:0' shape=(None,) dtype=int64>]


  6/337 [..............................] - ETA: 3:22 - loss: 10.3717 - activation_loss: 5.3102 - activation_1_loss: 5.0615WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0090s vs `on_train_batch_end` time: 9.4542s). Check your callbacks.


337/337 [==============================] - ETA: 0s - loss: 3.0375 - activation_loss: 1.5763 - activation_1_loss: 1.4612

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 384) dtype=int64>]



epoch=1, exact match score=0.76
start f score

epoch=1, f score=0.87
337/337 [==============================] - 391s 884ms/step - loss: 3.0375 - activation_loss: 1.5763 - activation_1_loss: 1.4612
Epoch 2/3
337/337 [==============================] - ETA: 0s - loss: 1.7699 - activation_loss: 0.9358 - activation_1_loss: 0.8341
epoch=2, exact match score=0.78
start f score

epoch=2, f score=0.88
337/337 [==============================] - 237s 704ms/step - loss: 1.7699 - activation_loss: 0.9358 - activation_1_loss: 0.8341
Epoch 3/3
337/337 [==============================] - ETA: 0s - loss: 1.3278 - activation_loss: 0.7119 - activation_1_loss: 0.6159
epoch=3, exact match score=0.78
start f score

epoch=3, f score=0.88
337/337 [==============================] - 237s 703ms/step - loss: 1.3278 - activation_loss: 0.7119 - activation_1_loss: 0.6159


**Saving Fscore and Exact Match**

In [ ]:
epoch = [1,2,3]

In [ ]:
f_score_list

[0.8702146392325509, 0.8777274165205987, 0.8810153318074356]

In [ ]:
exact_match_score_list

[0.763527248088278, 0.7786274320007743, 0.7819184977252928]

In [ ]:
import pandas as pd
d = {'exact_match': exact_match_score_list, 'f_score': f_score_list, 'epoch':epoch}


In [ ]:
df = pd.DataFrame(d)

In [ ]:
df

,exact_match,f_score,epoch
0,0.763527,0.870215,1
1,0.778627,0.877727,2
2,0.781918,0.881015,3


In [ ]:
df.to_csv('./content/drive/squad_bert_256.csv', index=False)

In [ ]:
#train_squad_examples[0]

In [ ]:
#x_temp = [x_eval[0][0:1], x_eval[1][0:1], x_eval[2][0:1]]

In [ ]:
#tans = model.predict(x_temp)

In [ ]:
#tans[0]

In [ ]:
'''
def printSquad(squad_input):
  print(squad_input.question)
  print(squad_input.context)
  print(squad_input.start_char_idx)
  print(squad_input.answer_text)
  print(squad_input.all_answers)
  print(squad_input.skip)
  print(squad_input.context_token_to_char)
printSquad(eval_squad_examples[0])
'''

'\ndef printSquad(squad_input):\n  print(squad_input.question)\n  print(squad_input.context)\n  print(squad_input.start_char_idx)\n  print(squad_input.answer_text)\n  print(squad_input.all_answers)\n  print(squad_input.skip)\n  print(squad_input.context_token_to_char)\nprintSquad(eval_squad_examples[0])\n'

In [ ]:
#start = np.argmax(tans[0])

In [ ]:
#start

In [ ]:
#end = np.argmax(tans[1])

In [ ]:
#end

In [ ]:
#offsets = eval_squad_examples[0].context_token_to_char

In [ ]:
'''
if start >= len(offsets):
  print("NO")
'''

'\nif start >= len(offsets):\n  print("NO")\n'

In [ ]:
'''
pred_char_start = offsets[start][0]
if end < len(offsets):
    pred_char_end = offsets[end][1]
    pred_ans = eval_squad_examples[0].context[pred_char_start:pred_char_end]
else:
    pred_ans = eval_squad_examples[0].context[pred_char_start:]
'''

'\npred_char_start = offsets[start][0]\nif end < len(offsets):\n    pred_char_end = offsets[end][1]\n    pred_ans = eval_squad_examples[0].context[pred_char_start:pred_char_end]\nelse:\n    pred_ans = eval_squad_examples[0].context[pred_char_start:]\n'

In [ ]:
#pred_ans